In [ ]:
conditions_df = pd.read_csv("competition_dataset/env_parameter_sample.csv")
conditions_df

In [ ]:
for j,row in conditions_df.iterrows():
    
    if j == 0:
        continue
    
    else:
        if row["sample_name"] == "No_sample":
            if row["time"][:15] == conditions_df.iloc[j-1]["time"][:15]:
                if int(row["time"][-1]) == int(conditions_df.iloc[j-1]["time"][-1]) + 1:
                    conditions_df.loc[j,"sample_name"] = conditions_df.iloc[j-1]["sample_name"]
        


In [ ]:
import torch
import math

In [ ]:
class Dataset():
    def __init__(self, species_df, conditions):
        self.species_df = species_df
        self.conditions = conditions

        #diferent species
        self.species_vocab = species_df["Specie"].unique().tolist()

    
    def __len__(self):
        return len(self.species_df)*3 #because 3 timesteps
    
    def __getitem__(self, idx):
        assert ((idx >= 0) & (idx < self.__len__())), "Dataset index out of range"

        real_id = int(math.ceil((idx+1)/3))
        t = 2 - (real_id*3 - (idx+1) ) +1
        
        real_id -= 1 #fix
        
        #specie one hot
        spc_name = self.species_df.iloc[real_id]["Specie"]
        one_hot_idx = self.species_vocab.index(spc_name)
        y = self.species_df.iloc[real_id]["t"+str(t)]
        
        return self.species_df.iloc[real_id]["id"] , torch.tensor([one_hot_idx],dtype=int) , torch.tensor(conditions[ str(self.species_df.iloc[real_id]["System"]) + str(self.species_df.iloc[real_id]["Treatment"]) ],dtype=torch.float) , torch.tensor(y,dtype=torch.float)

In [ ]:
#wrapper dataset
class train_dataset():
    def __init__(self,dataset):
        self.dataset = dataset
        self.lenght = len(dataset) - int(len(dataset)/4)

    def __len__(self):
        return self.lenght

    def __getitem__(self,idx):
        assert ((idx >= 0) & (idx < self.lenght)), "Dataset index out of range"
        return self.dataset.__getitem__(idx)

class test_dataset():
    def __init__(self,dataset):
        self.dataset = dataset
        self.lenght = int(len(dataset)/4)
        self.offset = int(len(dataset)/4)*3
    
    def __len__(self):
        return self.lenght

    def __getitem__(self,idx):
        assert ((idx >= 0) & (idx < self.lenght)), "Dataset index out of range"
        return self.dataset.__getitem__(idx+self.offset)